# Install Dependencies

In [1]:
%pip install langchain_openai==0.3.2 langgraph==0.2.68 langchain-community==0.3.16 langchain-experimental==0.3.4 langchain-core==0.3.32 ipython==8.31.0 pydantic-settings==2.7.1 httpx==0.28.1

Note: you may need to restart the kernel to use updated packages.


# Setup Environment

In [46]:
import os
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_postgres import PGVector

os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

CONNECTION = os.getenv('CONNECTION')
COLLECTION_NAME = os.getenv('COLLECTION_NAME')

llm = ChatOpenAI(model='gpt-4o-mini')
embeddings = OpenAIEmbeddings(model='text-embedding-3-small')

vector_store = PGVector(
    embeddings=embeddings,
    collection_name=COLLECTION_NAME,
    connection=CONNECTION,
    use_jsonb=True,
)

# Setup Tools

In [45]:
import httpx
from typing import Optional, Type

from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from pydantic_settings import BaseSettings
from langchain_postgres import PGVector
from langchain import hub


class APIToolSchema(BaseModel):
    path: str = Field(default="", description="the api path")
    query_params: Optional[dict] = Field(
        default=None, description="Optional search parameters"
    )


class RAGToolSchema(BaseModel):
    question: str = Field(default="", description="question to perform similarity search in vector store")


class APITool(BaseTool, BaseSettings):
    name: str = "api_tool"
    description: str = """Tool that calls GET on <https://mlb-player-api-924437296962.us-central1.run.app/players> apis."""
    args_schema: Type[APIToolSchema] = APIToolSchema
    base_url: str = "https://mlb-player-api-924437296962.us-central1.run.app/players"

    def _run(
            self,
            path: str = "",
            query_params: Optional[dict] = None,
            run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> dict:
        """Run the tool"""
        result = httpx.Client().get(self.base_url)

        print(result)
        return result.json()

    async def _arun(
            self,
            path: str = "",
            query_params: Optional[dict] = None,
            run_manager: Optional[AsyncCallbackManagerForToolRun] = None,
    ) -> dict:
        """Run the tool asynchronously."""

        async with httpx.AsyncClient() as client:
            result = await client.get(self.base_url)
            return result.json()


class RAGTool(BaseTool, BaseSettings):
    name: str = "rag_tool"
    description: str = """Tool that gets the most similar documents from the vector database."""
    args_schema: Type[RAGToolSchema] = RAGToolSchema

    def __init__(self):
        super().__init__()
        # self.vector_store = PGVector(
        #     embeddings=embeddings,
        #     collection_name=collection_name,
        #     connection=connection_string,
        #     use_jsonb=True,
        # )
        # self.prompt = hub.pull('rlm/rag-prompt')

    def _run(
            self,
            question: str,
            run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> dict:
        """Run the tool"""
        # retrieved_docs = self.vector_store.similarity_search(question)
        # docs_content = '\n\n'.join(doc.page_content for doc in retrieved_docs)
        # messages = self.prompt.invoke({'question': question, 'context': docs_content})
        # response = llm.invoke(messages)
        #
        # return response.content
        return ""

    async def _arun(
            self,
            question: str,
            run_manager: Optional[AsyncCallbackManagerForToolRun] = None,
    ) -> dict:
        """Run the tool asynchronously."""

        # retrieved_docs = self.vector_store.similarity_search(question)
        # docs_content = '\n\n'.join(doc.page_content for doc in retrieved_docs)
        # messages = self.prompt.invoke({'question': question, 'context': docs_content})
        # response = llm.invoke(messages)
        #
        # return response.content
        return ""


api_tool = APITool()
rag_tool = RAGTool()
    # connection_string=CONNECTION, collection_name=COLLECTION_NAME)

ValidationError: 2 validation errors for RAGTool
vector_store
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing
prompt
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing

In [15]:
players = api_tool.run({})
print(players)

<Response [200 OK]>
[{'id': 1, 'name': 'Juan Soto', 'age': 26, 'contract_start_date': '2024-12-11', 'contract_end_date': '2039-12-10', 'team': 'New York Mets', 'country': 'Dominican Republic', 'salary': '51 million per year'}, {'id': 2, 'name': 'Brent Rooker', 'age': 30, 'contract_start_date': '2025-01-07', 'contract_end_date': '2030-01-06', 'team': 'Oakland Athletics', 'country': 'United States', 'salary': '12 million per year'}, {'id': 3, 'name': 'Blake Snell', 'age': 32, 'contract_start_date': '2025-01-01', 'contract_end_date': '2029-12-31', 'team': 'Los Angeles Dodgers', 'country': 'United States', 'salary': '36.4 million per year'}, {'id': 4, 'name': 'Paul Goldschmidt', 'age': 37, 'contract_start_date': '2024-12-30', 'contract_end_date': '2025-12-29', 'team': 'New York Yankees', 'country': 'United States', 'salary': '12.5 million per year'}, {'id': 5, 'name': 'Reynaldo López', 'age': 31, 'contract_start_date': '2023-11-20', 'contract_end_date': '2026-11-19', 'team': 'Atlanta Brave

# Testing

In [6]:
def my_decorator(func):
    def wrapper():
        print("Something before the function runs.")
        func()
        print("Something after the function runs.")

    return wrapper


@my_decorator
def say_hello():
    print("Hello!")


say_hello()

Something before the function runs.
Hello!
Something after the function runs.
